tag_clustering.ipynb
- playlist_processing.ipynb를 통해 각 노래마다의 태그들에 대해 K-means 클러스터링을 진행해 같은 분위기의 노래를 군집화한다.

- input: processed_song.json
- output: developing

In [139]:
import json, sqlite3
import pandas as pd
import numpy as np
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import normalize
from sklearn.cluster import KMeans

In [144]:
class Music_Labeler:
    
    # Load music Dataframe and take most common k tags
    def __init__(
            self, 
            PATH = './json_data/processed_song.json',
            most_common_k = 100,
        ):
        
        with open(PATH, 'r', encoding="UTF8") as f:
            data = json.load(f)
        self.df = pd.json_normalize(data)
        
        # take only most common k tags
        common_tags = []
        for idx, row in self.df.iterrows():
            common_tags.extend(row['tags'].split())
        self.tags = [tag for [tag, cnt] in Counter(tags).most_common(most_common_k)]
        
        # except tags not in most common k tags each music 
        for idx, row in self.df.iterrows():
            self.df.loc[idx, 'new_tags'] = ' '.join([tag for tag in row['tags'].split() if tag in self.tags])
        self.df.drop('tags', axis=1, inplace=True)  
    
    def run(self):
        self.clustering_with_kmeans()
        self.result2db()
        
    def clustering_with_kmeans(self, n_clusters = 50):
        text = self.df['new_tags'].to_list()
        vectorizer = TfidfVectorizer(min_df = 5)
        X = normalize(vectorizer.fit_transform(text))
        
        # K-means model with @n_clusters clusters
        model = KMeans(n_clusters=n_clusters, random_state=10)
        self.df['label'] = model.fit_predict(X)
        
        # details(tags) each cluster 
        self.cluster_details = {}
        center_feature_idx = model.cluster_centers_.argsort()[:, ::-1]
        feature_names = vectorizer.get_feature_names_out()
        
        for cluster_num in range(n_clusters):
            self.cluster_details[cluster_num] = {}
            self.cluster_details[cluster_num]['cluster'] = cluster_num
            
            top_ftr_idx = center_feature_idx[cluster_num, :10]
            top_ftr = [feature_names[idx] for idx in top_ftr_idx]
            top_ftr_val = model.cluster_centers_[cluster_num, top_ftr_idx].tolist()
            
            self.cluster_details[cluster_num]['top_features'] = top_ftr
            self.cluster_details[cluster_num]['top_features_value'] = top_ftr_val
    
    def result2db(self, DB = './flask_api/db.db'):
        try:
            conn = sqlite3.connect(DB)
            cur = conn.cursor()
        except:
            print("DB Connection Error!")
            return
        
        try:
            cur.execute("DELETE FROM CLUSTER")
            for key, tags in self.cluster_details.items():
                for tag in tags['top_features']:
                    cur.execute("INSERT INTO CLUSTER VALUES (?, ?)", (key, tag))
        except:
            print("Failed to execute query")
            return
        finally:        
            conn.commit()
            conn.close()
        

In [ ]:
labeler = Music_Labeler()
labeler.clustering_with_kmeans()
labeler.cluster_details

In [140]:
detail = labeler.cluster_details

In [143]:
for key, tags in detail.items():
    for tag in tags['top_features']:
        print(key, tag)

0 듀엣
0 설렘
0 사랑
0 연인
0 기분전환
0 발라드
0 휴식
0 고백
0 힐링
0 이별
1 인디음악
1 인디
1 잔잔한
1 감성
1 새벽
1 카페
1 카페음악
1 힐링
1 휴식
1 저녁
2 스트레스
2 기분전환
2 힙합
2 드라이브
2 감성
2 휴식
2 매장음악
2 신나는
2 사랑
2 힐링
3 눈물
3 이별
3 슬픔
3 발라드
3 감성
3 새벽
3 그리움
3 추억
3 잔잔한
3 외로움
4 슬픔
4 이별
4 발라드
4 새벽
4 가을
4 감성
4 외로움
4 잔잔한
4 사랑
4 인디
5 힙합
5 감성힙합
5 알앤비
5 rnb
5 감성
5 드라이브
5 기분전환
5 새벽
5 발라드
5 인디
6 아이돌
6 걸그룹
6 숨은명곡
6 노래
6 발라드
6 감성
6 아이돌의숨은명곡
6 기분전환
6 추억
6 신나는
7 설렘
7 사랑
7 기분전환
7 카페
7 잔잔한
7 인디
7 감성
7 드라이브
7 새벽
7 발라드
8 분위기
8 rnb
8 카페
8 감성
8 알앤비
8 가을
8 드라이브
8 잔잔한
8 그루브
8 인디
9 회상
9 추억
9 힐링
9 휴식
9 슬픔
9 이별
9 기분전환
9 새벽
9 발라드
9 잔잔한
10 잔잔한
10 인디
10 감성
10 새벽
10 카페
10 혼자
10 휴식
10 발라드
10 어쿠스틱
10 기분전환
11 인디
11 드라이브
11 밴드
11 가요
11 기분전환
11 숨은명곡
11 카페
11 주말
11 달달한
11 노래
12 시원한
12 댄스
12 여름
12 신나는
12 기분전환
12 드라이브
12 여행
12 걸그룹
12 아이돌
12 스트레스
13 센치
13 트렌디
13 알앤비
13 휴식
13 감성
13 분위기
13 혼자
13 잔잔한
13 새벽
13 버스
14 국내힙합
14 힙합
14 드라이브
14 매장음악
14 감성힙합
14 스트레스
14 알앤비
14 매장
14 기분전환
14 rnb
15 새벽감성
15 새벽
15 잔잔한
15 휴식
15 감성
15 발라드
15 카페
15 힐링
15 산책
15 인디
16 힐링
16 휴식
16 새벽
16 기분전환
16 잔잔